<a href="https://colab.research.google.com/github/terkoizmy/TweetDisaster/blob/main/Tweet_Disaster_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare library

In [1]:
# download text hamer if you dont have it in your device
!pip install text_hammer

     |████████████████████████████████| 122kB 5.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
# Import library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import text_hammer as th
import re
import nltk
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras import layers, Input
from tensorflow.keras.optimizers import Adam, SGD
from keras.losses import BinaryCrossentropy
from keras.layers import Dense, LSTM, Dropout, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from sklearn.model_selection import train_test_split
from gensim.scripts.glove2word2vec import glove2word2vec
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import random
%matplotlib inline
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Download Glove for embeding

In [3]:
# Download Glove file
!wget http://nlp.stanford.edu/data/glove.6B.zip
# Unzip file
!unzip glove.6B.zip

--2021-05-26 00:49:22--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-05-26 00:49:22--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-05-26 00:49:23--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

# Prepare dataset

In [4]:
df_train = pd.read_csv('train.csv')

In [5]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
df_train.text

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

## Text processing

In [7]:
# Text Processing
def preprocess_text(texts):
  corpus = []
  for text in texts:
    text = re.sub(r'https?://\S+|www\.\S+|http?://\S+','',text)
    text = re.sub("["
                           u"\U0001F600-\U0001F64F"  # removal of emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           "]+",' ',text)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    wl = WordNetLemmatizer()
    all_stopwords = stopwords.words('english')
    text = [wl.lemmatize(word) for word in text if not word in set(all_stopwords)]
    text = [i for i in text if len(i)>2]
    text = ' '.join(text)
    corpus.append(text)

  return corpus

In [8]:
# Use text Processing
df_train['processed_text'] = preprocess_text(df_train['text'])

In [9]:
print(df_train['processed_text'][1:5])
print(df_train['text'][1:5])

1                   forest fire near ronge sask canada
2    resident asked shelter place notified officer ...
3    people receive wildfire evacuation order calif...
4    got sent photo ruby alaska smoke wildfire pour...
Name: processed_text, dtype: object
1               Forest fire near La Ronge Sask. Canada
2    All residents asked to 'shelter in place' are ...
3    13,000 people receive #wildfires evacuation or...
4    Just got sent this photo from Ruby #Alaska as ...
Name: text, dtype: object


## Split dataset

In [13]:
# Split data
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df_train['processed_text'], df_train['target'], test_size = 0.20)

In [14]:
print(X_train[3])

people receive wildfire evacuation order california


## Declare the vocabulary size for word embedding

In [15]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
top_words = 1000
t = Tokenizer(num_words=top_words) # num_words -> Vocablury size
t.fit_on_texts(X_train.tolist())


## Generate the word index for train and val data

In [16]:
X_train = t.texts_to_sequences(X_train.tolist())
X_val = t.texts_to_sequences(X_val.tolist())

In [17]:
print(X_train[3])

[56, 165, 518]


## Pading the sequence of tokens

In [18]:
from tensorflow.python.keras.preprocessing import sequence
max_review_length = 50
X_train = sequence.pad_sequences(X_train,maxlen=max_review_length,padding='post')
X_val = sequence.pad_sequences(X_val, maxlen=max_review_length, padding='post')

In [19]:
print(X_train[3])

[ 56 165 518   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


# Embeding Text

## Convert glove to Word2Vec

In [20]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec("glove.6B.50d.txt", "glove.6B.50d.txt.word2vec") 
# param 1: Glove file, param 2: Glove fileName for word2vec file 

(400000, 50)

## pretrained model glove

In [21]:
from gensim.models import Word2Vec, KeyedVectors
glove_model = KeyedVectors.load_word2vec_format("glove.6B.50d.txt.word2vec", binary=False)

In [22]:
#Embedding length based on selected model - we are using 50d here.
embedding_vector_length = 50

#Initialize embedding matrix
embedding_matrix = np.zeros((top_words + 1, embedding_vector_length))
print(embedding_matrix.shape)

(1001, 50)


In [23]:
for word, i in sorted(t.word_index.items(),key=lambda x:x[1]):
    if i > (top_words+1):
        break
    try:
        embedding_vector = glove_model[word] #Reading word's embedding from Glove model for a given word
        embedding_matrix[i] = embedding_vector
    except:
        pass

In [24]:
embedding_matrix[3]

array([ 3.68079990e-01,  2.08340004e-01, -2.23189995e-01,  4.62829992e-02,
        2.00979993e-01,  2.75150001e-01, -7.71269977e-01, -7.68040001e-01,
       -3.48610014e-01,  5.06200016e-01, -2.44010001e-01,  7.17750013e-01,
       -3.33480000e-01,  3.75539988e-01,  4.47560012e-01,  3.66979986e-01,
        4.35330003e-01,  4.75699991e-01, -5.61130010e-02, -9.35310006e-01,
       -2.75909990e-01,  3.16100001e-01,  2.21159995e-01,  3.63040000e-01,
        1.07570000e-01, -1.76380002e+00, -1.26240003e+00,  3.02839994e-01,
        5.62860012e-01, -1.02139997e+00,  3.23530006e+00,  4.84829992e-01,
        2.79530007e-02,  3.60819995e-02, -7.85539970e-02,  1.87610000e-01,
       -5.25730014e-01,  3.72000001e-02,  2.75790006e-01, -7.73599967e-02,
       -2.79549986e-01,  7.97519982e-01,  1.60279998e-03,  4.54789996e-01,
        8.83819997e-01,  4.38930005e-01, -1.92629993e-01, -6.72360003e-01,
       -3.97089988e-01,  2.51830012e-01])

# Create Model

In [25]:
#Initialize model
import tensorflow as tf
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

## input embedding layer

In [26]:
model.add(tf.keras.layers.Embedding( top_words+ 1, #Vocablury size
                                    embedding_vector_length, #Embedding size
                                    weights=[embedding_matrix], #Embeddings taken from pre-trained model
                                    trainable=False, #As embeddings are already available, we will not train this layer. It will act as lookup layer.
                                    input_length=max_review_length) #Number of words in each review
         )

## imput LSTM layer and optimizer

In [27]:
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(256)) #RNN State - size of cell state and hidden state
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            50050     
_________________________________________________________________
dropout (Dropout)            (None, 50, 50)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               314368    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 364,675
Trainable params: 314,625
Non-trainable params: 50,050
_________________________________________________________________


## Train model

In [29]:
model.fit(X_train,y_train,
          epochs=10,
          batch_size=32,          
          validation_data=(X_val, y_val))

Epoch 1/10
191/191 [==============================] - 36s 175ms/step - loss: 0.6184 - accuracy: 0.6658 - val_loss: 0.5406 - val_accuracy: 0.7584
Epoch 2/10
191/191 [==============================] - 33s 171ms/step - loss: 0.5176 - accuracy: 0.7603 - val_loss: 0.5389 - val_accuracy: 0.7774
Epoch 3/10
191/191 [==============================] - 33s 174ms/step - loss: 0.5111 - accuracy: 0.7635 - val_loss: 0.4977 - val_accuracy: 0.7768
Epoch 4/10
191/191 [==============================] - 33s 173ms/step - loss: 0.4915 - accuracy: 0.7708 - val_loss: 0.4892 - val_accuracy: 0.7663
Epoch 5/10
191/191 [==============================] - 33s 172ms/step - loss: 0.4799 - accuracy: 0.7742 - val_loss: 0.5150 - val_accuracy: 0.7722
Epoch 6/10
191/191 [==============================] - 32s 170ms/step - loss: 0.4803 - accuracy: 0.7806 - val_loss: 0.4768 - val_accuracy: 0.7814
Epoch 7/10
191/191 [==============================] - 33s 171ms/step - loss: 0.4671 - accuracy: 0.7836 - val_loss: 0.4849 - val_ac

In [30]:
val_pred=model.predict(X_val)
val_pred = np.where(val_pred > 0.5,1,0)

In [31]:
val_pred

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_val, val_pred))

              precision    recall  f1-score   support

           0       0.78      0.89      0.83       872
           1       0.82      0.66      0.73       651

    accuracy                           0.79      1523
   macro avg       0.80      0.78      0.78      1523
weighted avg       0.80      0.79      0.79      1523



In [33]:
# Store the output of lstm
x = model.get_layer('lstm').output
model2 = tf.keras.Model(model.input, x)

# Full train

In [34]:
df_test = pd.read_csv('test.csv')

In [35]:
df_test['processed_text'] = preprocess_text(df_test['text'])

In [36]:
### Declare the vocabulary size for word embedding
t = Tokenizer(num_words=top_words) # num_words -> Vocablury size
t.fit_on_texts(df_train['processed_text'].tolist())
train_seq  = t.texts_to_sequences(df_train['processed_text'].tolist())
test_seq  = t.texts_to_sequences(df_test['processed_text'].tolist())

train_seq = sequence.pad_sequences(train_seq,maxlen=max_review_length,padding='post')
test_seq = sequence.pad_sequences(test_seq, maxlen=max_review_length, padding='post')

In [37]:
train_seq

array([[446, 156,  64, ...,   0,   0,   0],
       [101,   1, 140, ...,   0,   0,   0],
       [447, 302, 157, ...,   0,   0,   0],
       ...,
       [607, 380,   0, ...,   0,   0,   0],
       [ 18, 963, 258, ...,   0,   0,   0],
       [125,  17, 404, ...,   0,   0,   0]], dtype=int32)

In [38]:
model.fit(train_seq,df_train['target'],
          epochs=25,
          batch_size=32)

Epoch 1/25
238/238 [==============================] - 38s 160ms/step - loss: 0.6657 - accuracy: 0.5868
Epoch 2/25
238/238 [==============================] - 38s 160ms/step - loss: 0.6346 - accuracy: 0.6473
Epoch 3/25
238/238 [==============================] - 39s 163ms/step - loss: 0.6030 - accuracy: 0.6774
Epoch 4/25
238/238 [==============================] - 39s 164ms/step - loss: 0.5642 - accuracy: 0.7148
Epoch 5/25
238/238 [==============================] - 39s 164ms/step - loss: 0.5384 - accuracy: 0.7400
Epoch 6/25
238/238 [==============================] - 39s 166ms/step - loss: 0.5236 - accuracy: 0.7466
Epoch 7/25
238/238 [==============================] - 40s 168ms/step - loss: 0.4988 - accuracy: 0.7622
Epoch 8/25
238/238 [==============================] - 40s 167ms/step - loss: 0.4829 - accuracy: 0.7766
Epoch 9/25
238/238 [==============================] - 40s 167ms/step - loss: 0.4658 - accuracy: 0.7890
Epoch 10/25
238/238 [==============================] - 38s 160ms/step - l

In [39]:
test_pred=model.predict(test_seq)

In [40]:
test_pred

array([[0.7546619 ],
       [0.71892893],
       [0.04489338],
       ...,
       [0.22225174],
       [0.8334807 ],
       [0.10308999]], dtype=float32)

In [41]:
df_test['target']=np.where(test_pred > 0.5,1,0)

In [42]:
df_test['target']

0       1
1       1
2       0
3       1
4       1
       ..
3258    1
3259    1
3260    0
3261    1
3262    0
Name: target, Length: 3263, dtype: int64

In [43]:
submit = pd.DataFrame()
submit['id'] = df_test['id']
submit['target'] = df_test['target']

In [44]:
submit

,id,target
0,0,1
1,2,1
2,3,0
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,0
3261,10874,1


# save to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
submit.to_csv('/content/gdrive/MyDrive/Submission/submit_LSTM.csv',index=False)